In [4]:
! head -1 ../partial_thread/target.fasta

>Q97AX4


In [5]:
! cat notes.txt

nucleophile is 380 
acid/base is 205 
tyr is 315 
ligand is 482 


In [10]:
# params for this protein 

params = { 
    'handle': 'Q97AX4',
    'nucleophile': 380,
    'acid_base': 205, 
    'backup': 315, 
    'ligand': 482, 
}

In [11]:
# set up the input files 

In [12]:
# puzzle setup 

puz_setup = 'version: 1\n{{\n"backbone_locked": "{}||"\n}}'
puz_fn = params['handle'] + '.puzzle_setup'
with open( puz_fn, 'w' ) as fn:
    fn.write( puz_setup.format( params[ 'ligand' ] ) ) 

In [13]:
! cat *puzzle_setup

version: 1
{
"backbone_locked": "482||"
}

In [14]:
# constraint file 

cst_template = '''# chemical restraints for pNPG hydrolysis by {handle}

# pNPG (residue {ligand}) and acid/base E{acid_base}
AtomPair O2 {ligand} OE2 {acid_base} HARMONIC 1.8 0.18
Angle O2 {ligand} OE2 {acid_base} CD {acid_base} HARMONIC 0 10
Dihedral O2 {ligand} OE2 {acid_base} CD {acid_base} CG {acid_base} HARMONIC 0 10

# pNPG and nucleophile E{nucleophile}
AtomPair C5 {ligand} OE2 {nucleophile} HARMONIC 2.0 0.20
Angle O2 {ligand} C5 {ligand} OE2 {nucleophile} HARMONIC 0 10
Angle C5 {ligand} OE2 {nucleophile} CD {nucleophile} HARMONIC 0 10
Dihedral C5 {ligand} OE2 {nucleophile} CD {nucleophile} CG {nucleophile} HARMONIC 0 10

# nucleophile E{nucleophile} and Y{backup}
AtomPair OE2 {nucleophile} OH {backup} HARMONIC 3.0 0.30
'''

with open( '{}_pNPG.cnstr'.format( params[ 'handle' ] ), 'w' ) as fn:
    fn.write( cst_template.format( **params ) )

In [15]:
! cat *.cnstr

# chemical restraints for pNPG hydrolysis by Q97AX4

# pNPG (residue 482) and acid/base E205
AtomPair O2 482 OE2 205 HARMONIC 1.8 0.18
Angle O2 482 OE2 205 CD 205 HARMONIC 0 10
Dihedral O2 482 OE2 205 CD 205 CG 205 HARMONIC 0 10

# pNPG and nucleophile E380
AtomPair C5 482 OE2 380 HARMONIC 2.0 0.20
Angle O2 482 C5 482 OE2 380 HARMONIC 0 10
Angle C5 482 OE2 380 CD 380 HARMONIC 0 10
Dihedral C5 482 OE2 380 CD 380 CG 380 HARMONIC 0 10

# nucleophile E380 and Y315
AtomPair OE2 380 OH 315 HARMONIC 3.0 0.30


In [16]:
# enzyme design header for input PDB 

enzdes_header='''REMARK   0 NE TEMPLATE X LG1 1 MATCH MOTIF A GLU {nucleophile} 1
REMARK   0 NE TEMPLATE X LG1 1 MATCH MOTIF A GLU {acid_base} 2
REMARK   0 NE TEMPLATE A GLU {nucleophile} MATCH MOTIF A TYR {backup} 3
'''

with open( 'enzdes_header.txt', 'w' ) as fn:
    fn.write( enzdes_header.format( **params ) ) 

In [17]:
! cat enzdes_header.txt

REMARK   0 NE TEMPLATE X LG1 1 MATCH MOTIF A GLU 380 1
REMARK   0 NE TEMPLATE X LG1 1 MATCH MOTIF A GLU 205 2
REMARK   0 NE TEMPLATE A GLU 380 MATCH MOTIF A TYR 315 3


In [19]:
! cat enzdes_header.txt > input_pose.pdb 
! grep '^ATOM' ../hybridize/S_0065.pdb >> input_pose.pdb 
! grep '^HETATM' pnpg.pdb >> input_pose.pdb